In [305]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X = data.data
y = data.target  
X,x_t,y,y_t = train_test_split(X,y,test_size=0.3)


In [306]:
import numpy as np

class Layer:
    def __init__(self,n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.bias = np.zeros((1,n_neurons))
            
    def forward(self,X):
        self.inputs = X
        return np.dot(X,self.weights)+self.bias
    def backward(self,dvals):
        self.dbias = np.sum(dvals,axis=0,keepdims=True)
        self.dweights = np.dot(self.inputs.T,dvals)
        self.dinputs = np.dot(dvals,self.weights.T)
        
        return self.dinputs

class Relu:
    def __init__(self):
        pass
    def forward(self,X):
        self.inputs = X
        return np.maximum(X,0)
    def backward(self,dvals):
        dinputs = dvals.copy()
        dinputs[self.inputs <= 0] = 0
        return dinputs
class Sigmoid:
    def __init__(self):
        pass 
    def forward(self,X):
        self.inputs = X
        X = np.clip(X, -500, 500)
        self.output = 1 / (1 + np.exp(-X))
        return self.output
    def backward(self,dvals):
        self.dvals = dvals*(self.output*(1-self.output))
        return self.dvals
class LogLoss:
    def __init__(self):
        pass
    def loss(self, pred, true):
        true = true.reshape(-1,1)
        eps = 1e-7
        pred = np.clip(pred, eps, 1 - eps)
        return -np.mean(true*np.log(pred) + (1-true)*np.log(1-pred))
    def backward(self,pred,true):
        n = true.shape[0]
        true = true.reshape(-1, 1)
        eps = 1e-7
        pred = np.clip(pred, eps, 1 - eps)
        return (-(true / pred) + (1 - true) / (1 - pred))/n
        

In [307]:
class NN:
    def __init__(self,no_inputs,layers,lr,epoch):
        self.hidden_layers = []
        self.lr = lr
        self.epoch = epoch
        self.loss = LogLoss()
        curr_input_size = no_inputs
        for i in layers:
            l = Layer(curr_input_size,i)
            a = Relu()
            self.hidden_layers.append([l,a])
            curr_input_size = i
        op = Layer(curr_input_size,1)
        op_a = Sigmoid() 
        self.hidden_layers.append([op,op_a])
        
    def forward(self,X):
        curr_output = X
        
        for [layer,activation] in self.hidden_layers:
            curr_output = activation.forward(layer.forward(curr_output)) 
        return curr_output       
    
    def BGD(self,X,y):
        
        for i in range(self.epoch):
            curr_output = self.forward(X)
            dvals = self.loss.backward(curr_output,y)
            for [layer,activation] in reversed(self.hidden_layers):
                dvals = activation.backward(dvals)
                dvals = layer.backward(dvals)
                layer.weights -= self.lr*layer.dweights
                layer.bias -= self.lr*layer.dbias
    
    def pred(self,X):
        output = np.ndarray.flatten(self.forward(X))
        return [1 if i>0.5 else 0 for i in output]

In [308]:
model = NN(X.shape[1],[4,3],0.01,5000)
model.BGD(X,y)

In [309]:
prediction = model.pred(x_t)

In [310]:
from sklearn.metrics import accuracy_score

accuracy_score(prediction,y_t)

0.9064327485380117